0. Import file

In [1]:
from instance_pyfile import *
from support_pyfile import *
import time


1. Confige LLM

In [2]:
import google.generativeai as genai

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE"
    }, 
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE"
    },
]

genai.configure(api_key='YOUR_API_KEY')

model = genai.GenerativeModel(model_name='gemini-pro', safety_settings=safety_settings)

config = genai.GenerationConfig(temperature=1.0)


In [3]:
best_cur = 999999999
best_now = 999999990
cur_temp = 1.0

2. Sample prompt constructions in paper
- task_description: about TSP
- in_context: instances from the population.
- task_instruction: step-by-step instructions for LLM to follow

In [4]:
description = 'You are given a list of points with coordinates: {' + list2str() + '}. Your task is to find a trace, with the shortest possible length, that trverses each point exactly once.\n'

in_context = 'Below are some previous traces and their lengths. The traces are arranged in descending order based on their lengths, where lower values are better.\n'
#in_context += pool2examples(Pool)                  ! NOTICE that each generation, Pool change.

task_instruction = 'Please follow the instruction step-by-step to generate new trace:\n'
task_instruction += 'Step 1. Select two traces from the above traces.\n'
task_instruction += 'Step 2. Crossover the two traces chosen in Step 1 and generate a new trace.\n'
task_instruction += 'Step 3. Mutate the trace generated in Step 2 and generate a new trace.\n'
task_instruction += ('Step 4. Keep the generated trace generated in Step 3, repeat Step 1, 2, 3, until you have ' + str(N) + ' generated traces.\n')
task_instruction += 'Directly give me all the chosen traces at Step 1, bracketed them with <selection> and </selection>, and all the generated traces at Step 3, bracketed them with <res> and </res>. Not any explanation needed.'

3. Algorithm1 (from paper):
- each API call, provide all N child in the population as in-context learn for LLM
- for 1 call, LLM try to generate N new offspring from the provided population.
- Note with the Self-Adaption

In [5]:
Pool = randomFirstN(n, N)

g = 1

while g <= G :

    start_gene = time.time()

    prompt = description + (in_context + pool2examples(Pool)) + task_instruction

    listOff = []
    best_cur = Pool[N-1].length

    while len(listOff) < N :

        config = genai.GenerationConfig(temperature=cur_temp)

        start_call = time.time()

        response = model.generate_content(contents=prompt, generation_config=config)

        spent_call = time.time() - start_call

        print(f"Time for the API calls in generation {g}: {spent_call}")

        newGen = cutGenTrace(response.text)
        for s in newGen:
            if checkPermu(s, n) :
                listOff.append(s)
    #P_sharp = transform(listOff[:N])

    Pool = updatePool(Pool, listOff[:N], N)
    
    best_now = Pool[N-1].length

    if check_stuck(best_cur, best_now, K=10):
        cur_temp += 0.1
        if cur_temp == 2.0 :
            cur_temp = 1.5
            
        print("Self-Adaptation!")

    spent_gene = time.time() - start_gene
    print(f"Total time for the generation {g}: {spent_gene}")
    print(f"best solution now is: {best_now}")
    print(f"worst solution now is: {Pool[0].length}")

    g += 1

print(Pool[N-1])

Time for the API calls in generation 1: 17.390012741088867
Time for the API calls in generation 1: 24.922216176986694
The cur_0: [6, 9, 3, 7, 12, 11, 2, 8, 4, 10, 14, 5, 1, 13]
On other hand, the new_0: [11, 10, 1, 5, 8, 12, 9, 4, 14, 13, 3, 7, 2, 6]
The size of pool now is: 31
got stuck 1 time
Total time for the generation 1: 42.31222891807556
best solution now is: 5320.0
worst solution now is: 7031.0
Time for the API calls in generation 2: 25.223801374435425
Time for the API calls in generation 2: 22.859049558639526
The cur_0: [1, 2, 6, 5, 8, 12, 7, 13, 4, 9, 10, 14, 11, 3]
On other hand, the new_0: [11, 5, 1, 4, 6, 7, 12, 2, 10, 14, 13, 9, 3, 8]
The size of pool now is: 32
got update!
Total time for the generation 2: 48.0838520526886
best solution now is: 4642.0
worst solution now is: 6491.0
Time for the API calls in generation 3: 22.549835681915283
Time for the API calls in generation 3: 23.73498558998108
The cur_0: [1, 11, 6, 5, 9, 7, 10, 13, 12, 2, 14, 3, 4, 8]
On other hand, the

InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting